<a href="https://colab.research.google.com/github/abinstanislaus/abinstanislaus/blob/main/MyCurrentProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator as idg




In [6]:
drive.mount("/content/gdrive")
os.chdir("/content/gdrive/My Drive/FER2013")
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [7]:
img_size = 48
training_dir = ("/content/gdrive/My Drive/FER2013/train")
val_dir = ("/content/gdrive/My Drive/FER2013/test1")
train_datagenerator = idg(rescale = 1/255, shear_range=0.2, zoom_range=(0.6, 0.8), rotation_range=28.0, width_shift_range=0.25, height_shift_range=0.30, horizontal_flip = True, vertical_flip = True, fill_mode = "nearest")
val_datagenerator = idg(rescale = 1/255, shear_range=0.2, zoom_range=(0.6, 0.8), rotation_range=28.0, width_shift_range=0.25, height_shift_range=0.30, horizontal_flip = True, vertical_flip = True, fill_mode = "nearest" )

train_generator = train_datagenerator.flow_from_directory(directory = training_dir, target_size = (img_size,img_size), batch_size = 64, color_mode = "grayscale", class_mode = "categorical", subset = "training" )
validation_generator = val_datagenerator.flow_from_directory( directory = val_dir, target_size = (img_size,img_size), batch_size = 64, color_mode = "grayscale", class_mode = "categorical", subset = "validation" )



Found 2359 images belonging to 7 classes.
Found 0 images belonging to 7 classes.


In [8]:
#cALL BAcks

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs ={}):
    if(logs.get('accuracy')>0.95):
      print('\n Reached 95% accuracy so cancelling training!')
      self.model.stop_training = True


callbacks = myCallback()


In [9]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [10]:
#1st CNN layer
no_of_classes = 7
model = tf.keras.models.Sequential()
model.add(Conv2D(64,(3,3),padding="same",input_shape=(48,48,1)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))



#2nd CNN layer
model.add(Conv2D(128,(5,5),padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


#3rd CNN layer
model.add(Conv2D(512,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))



model.add(Dense(no_of_classes,activation="softmax"))

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])


history = model.fit_generator(train_generator,
                    epochs =10)
ig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()